## Compare images

**Pairwise image comparison demonstrating diverged analysis.**

This analysis cuts _n_ sub-images from a source image based on user specified Region of Interests (ROIs), then performs all pair-wise color-comparisons among these cut images and cluster the images based on color similarity. When a single ROI changes (is being dragged), *Quibbler* knows to only make the calculation needed: cutting this specific image (see a single reporting of: "Cutting image") and re-performing the pairwise comparisons of this image with all others (2 _n_ - 1 recalculations. See reporting of "Comparing images"). 

* **Features**
    * Diverged calculations of quib slices
    * Calling user function with np.vectorize
    * Graphics-driven assignments
    * Inverse assignments
    * Assignment template


* **Try me**
    * Drag each region of interest to define the cut images. Watch the number of re-calculations being made.
    * Change the similarity threshold either with the slider or on the color-scale. 
    * Add or remove images with the "Image count" slider.


In [1]:
# Imports:

import pyquibbler as qb
from pyquibbler import iquib, q, quiby
qb.override_all()

import numpy as np
from scipy.sparse.csgraph import connected_components
from functools import partial
from matplotlib import pyplot as plt, widgets
from mpl_toolkits.axes_grid1 import ImageGrid

%matplotlib tk

In [2]:
# Define vectorized analysis steps:

@partial(np.vectorize, signature='(4),()->()', pass_quibs=True, is_graphics=True)
def create_roi(roi, axs):
    rectprops = dict(facecolor='k', edgecolor='k', alpha=0.2, fill=True)
    widgets.RectangleSelector(axs, extents=roi, rectprops=rectprops)


@partial(np.vectorize, signature='(w,h,c),(4)->()',otypes=[object])
def cut_image(image, roi):
    print("Cutting image")
    return image[roi[2]:roi[3], roi[0]:roi[1]]


@partial(np.vectorize, otypes=[float])
def image_distance(img1, img2):
    print("Comparing images")
    rgd_distance = np.average(img1, axis=(0, 1)) - np.average(img2, axis=(0, 1))
    return np.linalg.norm(rgd_distance / 255)


@partial(np.vectorize, signature='(),()->()', is_graphics=True)
def show_cut_images(ax, img):
    ax.imshow(img)


@partial(np.vectorize, signature='(),()->()', is_graphics=True)
def plot_image_cluster_label(ax, index):
    ax.text(0, 0, chr(index+65), fontsize=20, ha='left', va='top')


In [3]:
@partial(np.vectorize, lazy=False)
def show_adjacency(axs, x, y, adjacent):
    symbol = 'x' if adjacent else '.'
    axs.plot(x, y, symbol, color='r')

In [4]:
# Read and draw source image
file_name = iquib('../data_files/pipes.jpg')
image = plt.imread(file_name)

plt.figure(1, figsize=[6, 6])
ax1 = plt.axes([0.15, 0.3, 0.7, 0.65])
ax1.imshow(image);

In [5]:
# Define ROIs:
images_count = iquib(6, assignment_template=(0, 10, 1))

roi_default = iquib([[20, 100, 20, 100]], allow_overriding=False)

rois = np.repeat(roi_default, images_count, axis=0)
rois.setp(assignment_template=(0, 1000, 1), allow_overriding=True)

similiarity_threshold = iquib(.1)

In [6]:
# Cut the images from image according to the rois
cut_images = cut_image(image, rois)

In [7]:
# Draw the rois
create_roi(rois, ax1);

In [8]:
# Add slides for similarity threshold and image count:
widgets.Slider(
    ax=plt.axes([0.4, 0.2, 0.4, 0.03]),
    label="Similiarity threshold",
    valmin=0, valmax=1, valstep=.05,
    valinit=similiarity_threshold)

widgets.Slider(
    ax=plt.axes([0.4, 0.1, 0.4, 0.03]),
    label="Image count",
    valmin=1, valmax=9, valstep=1,
    valinit=images_count);

In [9]:
# Figure 2 - Plot the cut images
fig = plt.figure(2)
grid_axes = iquib(ImageGrid(fig, 111, nrows_ncols=(3, 3), axes_pad=0.1))
show_cut_images(grid_axes[:images_count], cut_images);

Cutting image
Cutting image
Cutting image
Cutting image
Cutting image
Cutting image


In [10]:
# Calculate all pairwise image comparisons 
image_distances = image_distance(np.expand_dims(cut_images, 1), cut_images)
adjacents = image_distances < similiarity_threshold

In [11]:
# Figure 3 - Plot distance matrix
fig = plt.figure(3)
fig.clf()
axs = fig.add_axes([0.1, 0.15, 0.7, 0.7])
axs.imshow(1 - image_distances, cmap='gray', vmin=0, vmax=1) \
    .setp(graphics_update='drop')
axs.axis([-0.5, images_count-0.5, -0.5, images_count-0.5])
axs.set_title('pairwise distance between images')
axs.set_xlabel('Image number')
axs.set_ylabel('Image number')

image_nums = np.arange(images_count)
show_adjacency(axs, np.expand_dims(image_nums, 1), image_nums, adjacents) \
    .setp(graphics_update='drop');

# colormap
axclr = fig.add_axes([0.85, 0.15, 0.06, 0.7])
clrmap = np.linspace(1, 0, 10).reshape(10, 1)
axclr.imshow(clrmap, cmap='gray', vmin=0, vmax=1)
axclr.plot([-0.5, 0.5], similiarity_threshold * 10 - 0.5 + np.array([0, 0]), 
           '-r', linewidth=4, picker=True)
axclr.set_xticks([])
axclr.set_yticks([])
axclr.set_ylabel('Similarity Threshold');

Comparing images
Comparing images
Comparing images
Comparing images
Comparing images
Comparing images
Comparing images
Comparing images
Comparing images
Comparing images
Comparing images
Comparing images
Comparing images
Comparing images
Comparing images
Comparing images
Comparing images
Comparing images
Comparing images
Comparing images
Comparing images
Comparing images
Comparing images
Comparing images
Comparing images
Comparing images
Comparing images
Comparing images
Comparing images
Comparing images
Comparing images
Comparing images
Comparing images
Comparing images
Comparing images
Comparing images


In [12]:
# add cluster label
cluster_indeces = q(connected_components, adjacents)[1]
plot_image_cluster_label(grid_axes[:images_count], cluster_indeces) \
    .setp(graphics_update='drop')

np.vectorize(plot_image_cluster_label, (),()->())(grid_axes[:images_count], cluster_indeces)

In [13]:
pass